In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import r2_score

df = pd.read_csv('FinSalesPriceData_train.csv')


df_test = pd.read_csv('FinSalesPriceData_test.csv')
rival = pd.read_csv('CompetitorPriceData.csv')

df['calendar_day'] = pd.to_datetime(df['calendar_day'])
df_test['calendar_day'] = pd.to_datetime(df_test['calendar_day'])
rival['fiscal_week_start_date'] = pd.to_datetime(rival['fiscal_week_start_date'])







# df = pd.read_csv('/Users/clj/Desktop/capstone/BigW_df/FinSalesPriceData_train.csv')


# df_test = pd.read_csv('/Users/clj/Desktop/capstone/BigW_df/FinSalesPriceData_test.csv')
# rival = pd.read_csv('/Users/clj/Desktop/capstone/BigW_df/CompetitorPriceData.csv')

# df['calendar_day'] = pd.to_datetime(df['calendar_day'])
# df_test['calendar_day'] = pd.to_datetime(df_test['calendar_day'])
# rival['fiscal_week_start_date'] = pd.to_datetime(rival['fiscal_week_start_date'])

In [ ]:
df['gst_flag'] = df['gst_flag'].replace({'Y':1,'N':0})
df_test['gst_flag'] = df_test['gst_flag'].replace({'Y':1,'N':0})

### COGS function

In [ ]:
def calculate_cogs(sales_amount,gross_profit,gst_flag):
    sales_amount_ex_gst = sales_amount/(1+0.1*gst_flag)
    return (sales_amount_ex_gst - gross_profit)

### Gross profit function

In [ ]:
def calculate_gross_profit(unit_sold, promo_sales, regular_sales, cogs,gst_flag):
    gst = (0.1 if gst_flag == 1 else 0)
    sales_ex_gst = (promo_sales + regular_sales)/(1+ gst)
    return (sales_ex_gst - (unit_sold * cogs))

### Fill NA

In [ ]:
df['promo_price'] = np.where((df['sell_price'] == df['promo_price']) 
                             | (df['promo_price'].isnull()), 0, df['promo_price'])
df_test['promo_price'] = np.where((df_test['sell_price'] == df_test['promo_price']) 
                             | (df_test['promo_price'].isnull()), 0, df_test['promo_price'])

df['promo_sales'] = np.where((df['promo_sales'].isnull()), 0, df['promo_price'])
df_test['promo_sales'] = np.where((df_test['promo_sales'].isnull()), 0, df_test['promo_price'])

df['scanback'] = np.where((df['scanback'].isnull()), 0, df['promo_price'])
df_test['scanback'] = np.where((df_test['scanback'].isnull()), 0, df_test['promo_price'])

### FF filling dettol price

In [ ]:
# 筛选出 article_id 为 695147 的行索引
indices = df[(df['article_id'] == 695147) & (df['sales_units'] > 0)].index

# 对这些行计算 sell_price 的估算值
df.loc[indices, 'estimated_sell_price'] = df.loc[indices, 'sales_amount'] / df.loc[indices, 'sales_units']

# 对整个 DataFrame 应用 forward fill 来填补 estimated_sell_price
df['estimated_sell_price'] = df['estimated_sell_price'].fillna(method='ffill')

# 现在，您可以选择直接更新 sell_price 列
df.loc[df['article_id'] == 695147, 'sell_price'] = df.loc[df['article_id'] == 695147, 'estimated_sell_price']

# 如果不再需要 estimated_sell_price 列，可以选择删除它
df.drop(columns=['estimated_sell_price'], inplace=True)


### 移除Deranged 产品

In [ ]:
df.drop(df[(df['cnt_site_art_ranged'] == 0) & (df['sales_units'] != 0)].index, inplace=True)
df_test.drop(df_test[(df_test['cnt_site_art_ranged'] == 0) & (df_test['sales_units'] != 0)].index, inplace=True)


### 是否打折

In [ ]:
df['is_promo'] = np.where(df['promo_price'] != 0, 1, 0)
df_test['is_promo'] = np.where(df_test['promo_price'] != 0, 1, 0)


### 去掉负数 Gross profit 

In [ ]:
# Step 1: Identify article_ids (if you haven't already)
negative_profit = df.groupby(['category', 'subcategory','article_id']).agg(
    gross_profit=('gross_profit', 'sum'),  # Assuming the column name is 'gross_profits'
    sales_units=('sales_units', 'sum'),
).reset_index()

negative_profit_test = df_test.groupby(['category', 'subcategory','article_id']).agg(
    gross_profit=('gross_profit', 'sum'),  # Assuming the column name is 'gross_profits'
    sales_units=('sales_units', 'sum'),
).reset_index()


negative_profit_zero_sales_ids = negative_profit[
    (negative_profit['gross_profit'] <= 0) & (negative_profit['sales_units'] == 0)
]['article_id'].unique()

negative_profit_zero_sales_ids_test = negative_profit_test[
    (negative_profit_test['gross_profit'] <= 0) & (negative_profit_test['sales_units'] == 0)
]['article_id'].unique()

# Step 2: Find indexes of rows in df to drop
indexes_to_drop = df[df['article_id'].isin(negative_profit_zero_sales_ids)].index

indexes_to_drop_test = df_test[df_test['article_id'].isin(negative_profit_zero_sales_ids_test)].index

# Step 3: Drop these rows in-place from df
df.drop(indexes_to_drop, inplace=True)

df_test.drop(indexes_to_drop_test, inplace=True)


### merged

In [ ]:
### 合并competitor df
merged_df = pd.merge(df, rival, how='left', left_on=['article_id', 'calendar_day'], right_on=['article_id', 'fiscal_week_start_date'])

# Renaming the columns from the rival DataFrame to match your query output if necessary
merged_df.rename(columns={
    'competitor': 'competitor',
    'competitor_shelf_price': 'competitor_shelf_price',
    'competitor_total_price': 'competitor_total_price'
}, inplace=True)

# Ensure the DataFrame is sorted by 'article_id' and 'calendar_day' before forward filling
merged_df = merged_df.sort_values(by=['article_id', 'calendar_day'])

# Forward fill the missing values for the specified columns
columns_to_ffill = ['fiscal_week_start_date', 'competitor', 'competitor_shelf_price', 'competitor_total_price']
merged_df[columns_to_ffill] = merged_df.groupby('article_id')[columns_to_ffill].ffill()

merged_df.info()

In [ ]:
### 合并competitor df
merged_df_test = pd.merge(df_test, rival, how='left', left_on=['article_id', 'calendar_day'], right_on=['article_id', 'fiscal_week_start_date'])

# Renaming the columns from the rival DataFrame to match your query output if necessary
merged_df_test.rename(columns={
    'competitor': 'competitor',
    'competitor_shelf_price': 'competitor_shelf_price',
    'competitor_total_price': 'competitor_total_price'
}, inplace=True)

# Ensure the DataFrame is sorted by 'article_id' and 'calendar_day' before forward filling
merged_df_test = merged_df_test.sort_values(by=['article_id', 'calendar_day'])

# Forward fill the missing values for the specified columns
columns_to_ffill = ['fiscal_week_start_date', 'competitor', 'competitor_shelf_price', 'competitor_total_price']
merged_df_test[columns_to_ffill] = merged_df_test.groupby('article_id')[columns_to_ffill].ffill()

merged_df_test.info()

In [ ]:
# corr = sales_dummy.corr().round(3)
# corr_units = corr.loc['sales_units',:].sort_values(ascending=False)

In [ ]:
# best_para_index = corr_units.loc[(corr_units > 0.1) | (corr_units < -0.1)]
# best_para_index

In [ ]:
# merged_df.to_csv('/Users/clj/Desktop/capstone/BigW_df/bigw_merged.csv', index=False)

In [ ]:
merged_df

## EDA

In [ ]:
# Find the fluctuation of gross_profit 
sns.lineplot(data = train, x='calendar_day', y='gross_profit', color = 'steelblue')
plt.ylabel('gross_profit (AUD)')
plt.title('The fluctuation of gross_profit')
plt.savefig("gross_profit")

In [ ]:
# the correlation between the variables and gross_profit
train.corr()['gross_profit'].abs()

In [ ]:
# Calculate the correlation matrix
corr_matrix = train.corr()

# Select correlations of 'sales' with other variables
gross_profit_corr = corr_matrix['gross_profit']

# Create the heatmap
plt.figure(figsize=(10,8))
sns.heatmap(gross_profit_corr.to_frame(), cmap='crest', annot=True, fmt=".2f", cbar=True)
plt.title('Correlation of gross profit')
plt.savefig('corr of gross profit')

In [ ]:
# sales_amount * sales_unit
sns.scatterplot(data=train, x="sales_units", y="sales_amount", hue="category", legend="full")

In [ ]:
# 
sales_units1= trian.groupby('category')['sales_units'].sum()
promo_units1= train.groupby('category')['promo_units'].sum()


categories = ['Skin & Sun Care', 'Baby Consumables', 'Household Cleaning', 'Personal Hygiene']
sales_units = [6383005, 7850024, 11803738, 7811973]
promo_units = [3580535, 2643383, 3273582, 3140521]
df = pd.DataFrame({'Category': categories, 'Sales Units (%)': sales_units, 'promo_units (%)': promo_units})
fig, ax = plt.subplots(figsize=(10, 6))
df.plot(kind='bar', x='Category', ax=ax)
ax.set_ylabel('Values')
ax.set_title('Sales Data by Category')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# scanback Vs sales units
plt.scatter(train['scanback'], train['sales_units'], color ="steelblue", alpha = 0.8)
plt.ylabel('sales_units')
plt.xlabel('scanback (AUD)')
plt.title('sales_units Vs scanback')
plt.savefig('ss')

### Feature Engineering 

In [ ]:
# 独家VS非独家
merged_df['is_only_supplier'] = (merged_df['competitor'].notna()).astype(int)
merged_df_test['is_only_supplier'] = (merged_df_test['competitor'].notna()).astype(int)


# number of brands with subcategory (当下细分市场的竞争有多激烈)

# Step 1: Group by 'category' and 'subcategory' and count distinct brands
brand_count = merged_df.groupby(['category', 'subcategory'])['brand'].nunique().reset_index(name='brand_cnt')
brand_count_test = merged_df_test.groupby(['category', 'subcategory'])['brand'].nunique().reset_index(name='brand_cnt')

# Step 2: Rank the subcategories within each category based on brand count
brand_count['competition_level'] = brand_count.groupby('category')['brand_cnt'].rank(method='min', ascending=False)
brand_count_test['competition_level'] = brand_count_test.groupby('category')['brand_cnt'].rank(method='min', ascending=False)

# Merge the ranking back to the original DataFrame
merged_df = merged_df.merge(brand_count[['category', 'subcategory', 'competition_level']], on=['category', 'subcategory'], how='left')
merged_df_test = merged_df_test.merge(brand_count[['category', 'subcategory', 'competition_level']], on=['category', 'subcategory'], how='left')


In [ ]:
### weekly 处理

######## weekly sell_price and promo_price
# Creating a 'week' column that represents the week of the year
merged_df['week'] = merged_df['calendar_day'].dt.to_period('W')
merged_df_test['week'] = merged_df_test['calendar_day'].dt.to_period('W')

# # Group by 'article_id' and 'week', and calculate the mean for 'sell_price' and 'promo_price'
# weekly_avg_prices = merged_df.groupby(['article_id', 'week']).agg(
#     avg_price=('sell_price', 'mean'),
#     avg_promo_price=('promo_price', 'mean')
# ).reset_index()

# weekly_avg_prices_test = merged_df_test.groupby(['article_id', 'week']).agg(
#     avg_price=('sell_price', 'mean'),
#     avg_promo_price=('promo_price', 'mean')
# ).reset_index()

# merged_df = pd.merge(merged_df, weekly_avg_prices, on=['article_id', 'week'], how='left')
# merged_df_test = pd.merge(merged_df_test, weekly_avg_prices_test, on=['article_id', 'week'], how='left')

# Converting to string format
# weekly_avg_prices['week'] = weekly_avg_prices['week'].astype(str)



######## weekly一起促销产品的数量 within subcategory
on_sale = merged_df['is_promo'] == 1
on_sale_test = merged_df_test['is_promo'] == 1

# Step 2: Count distinct SKUs on sale per week and subcategory
competitive_count = merged_df[on_sale].groupby(['week', 'category','subcategory']).agg(
    num_skus_on_sale=('article_id', 'nunique')
).reset_index()

competitive_count_test = merged_df_test[on_sale].groupby(['week', 'category','subcategory']).agg(
    num_skus_on_sale=('article_id', 'nunique')
).reset_index()

# Step 3: Merge this count back to the original DataFrame
merged_df = pd.merge(merged_df, competitive_count, on=['week', 'subcategory'], how='left')
merged_df_test = pd.merge(merged_df_test, competitive_count_test, on=['week', 'subcategory'], how='left')

# Fill NaN values with 0 where there were no on-sale SKUs in the week and subcategory
merged_df['num_skus_on_sale'].fillna(0, inplace=True)
merged_df_test['num_skus_on_sale'].fillna(0, inplace=True)

######## (weekly sell - weekly promo)/weeklysell 打折力度
# merged_df['promo_power'] = (merged_df['avg_price'] - merged_df['avg_promo_price']) / merged_df['avg_price']
# merged_df_test['promo_power'] = (merged_df_test['avg_price'] - merged_df_test['avg_promo_price']) / merged_df_test['avg_price']

# # Handling cases where the weekly sell price might be zero to avoid division by zero errors
# merged_df['promo_power'] = merged_df['promo_power'].replace([float('inf'), -float('inf')], 0)
# merged_df_test['promo_power'] = merged_df_test['promo_power'].replace([float('inf'), -float('inf')], 0)



merged_df['promo_power'] = (merged_df['sell_price'] - merged_df['promo_price']) / merged_df['sell_price']
merged_df_test['promo_power'] = (merged_df_test['sell_price'] - merged_df_test['promo_price']) / merged_df_test['sell_price']
# Fill NaN values that might have occurred due to both prices being zero or missing promo prices
merged_df['promo_power'].fillna(0, inplace=True)
merged_df_test['promo_power'].fillna(0, inplace=True)








In [ ]:
# Mark the week of day for the given product in given date
merged_df['day_of_week'] = merged_df['calendar_day'].dt.day_of_week
merged_df_test['day_of_week'] = merged_df_test['calendar_day'].dt.day_of_week

# Detect whether the product was sold in weekend
merged_df['is_weekend'] = (merged_df['day_of_week'] >= 5).astype(int)
merged_df_test['is_weekend'] = (merged_df_test['day_of_week'] >= 5).astype(int)

In [ ]:
#promotion active flag
merged_df['promotion_active'] = np.where(merged_df['promo_price'] != 0, 1,0)
merged_df_test['promotion_active'] = np.where(merged_df_test['promo_price'] != 0, 1,0)

In [ ]:
#距离上次打折的时间
# Detect promotion end dates
merged_df['promo_ends'] = merged_df.groupby('article_id')['promotion_active'].diff().fillna(0).lt(0)
merged_df_test['promo_ends'] = merged_df_test.groupby('article_id')['promotion_active'].diff().fillna(0).lt(0)

In [ ]:
#function to calculate days since last promotion
def days_since_last_promo(row,group):
    prior_promo_ends = group.loc[(group['calendar_day'] < row['calendar_day']) & group['promo_ends'],'calendar_day']
    if not prior_promo_ends.empty:
        return (row['calendar_day'] - prior_promo_ends.max()).days
    return None

In [ ]:
#Apply the function to dataframe row_wise
#merged_df['days_since_last_promo'] = merged_df.apply(lambda row: days_since_last_promo(row,merged_df), axis=1)
#merged_df_test['days_since_last_promo'] = merged_df_test.apply(lambda row: days_since_last_promo(row,merged_df_test), axis=1)

#Mark products that have no promotion occurred before
#merged_df['days_since_last_promo'].fillna(-1,inplace=True) # Use -1 to indicate no prior promotion
#merged_df_test['days_since_last_promo'].fillna(-1,inplace=True)
# Too time consuming, don't run this part

In [ ]:
# days_since_last_promo = pd.read_csv('days_since_last_promo.csv')
# days_since_last_promo_test = pd.read_csv('days_since_last_promo_test.csv')
days_since_last_promo = pd.read_csv('days_since_last_promo.csv')
days_since_last_promo_test = pd.read_csv('days_since_last_promo_test.csv')

days_since_last_promo = days_since_last_promo.drop_duplicates()
days_since_last_promo_test = days_since_last_promo_test.drop_duplicates()

days_since_last_promo['calendar_day'] = pd.to_datetime(days_since_last_promo['calendar_day'])
days_since_last_promo_test['calendar_day'] = pd.to_datetime(days_since_last_promo_test['calendar_day'])
merged_df = pd.merge(merged_df, days_since_last_promo, on=['calendar_day', 'article_id'], how='left')
merged_df_test = pd.merge(merged_df_test,days_since_last_promo_test, on=['calendar_day','article_id'])


# 聚合 sales_units
sales_agg = merged_df.groupby(['article_id', 'calendar_day']).agg({'sales_units': 'sum'}).reset_index()
# 选择原始数据集中不想改变的列
original_columns = merged_df.drop(columns='sales_units').drop_duplicates()
# 左连接聚合后的数据
merged_df = pd.merge(original_columns, sales_agg, on=['article_id', 'calendar_day'], how='left').drop_duplicates(subset=['article_id', 'calendar_day'])


sales_agg = merged_df_test.groupby(['article_id', 'calendar_day']).agg({'sales_units': 'sum'}).reset_index()
original_columns = merged_df_test.drop(columns='sales_units').drop_duplicates()
merged_df_test = pd.merge(original_columns, sales_agg, on=['article_id', 'calendar_day'], how='left').drop_duplicates(subset=['article_id', 'calendar_day'])



In [ ]:
sell_price = np.array(merged_df_test['sell_price'])

In [ ]:

# Manually specify which columns are numerical
numerical_cols = ['sell_price', 'promo_price', 'cnt_site_art_ranged',
       'cnt_site_art_ranged_pstv_soh', 'tot_soh_ranged_sites','num_skus_on_sale','days_since_last_promo']  # Replace with your actual numerical columns
# Initialize the scaler
scaler = MinMaxScaler()


# Apply scaling only to the numerical columns
merged_df_test[numerical_cols] = scaler.fit_transform(merged_df_test[numerical_cols])
merged_df[numerical_cols] = scaler.fit_transform(merged_df[numerical_cols])



In [ ]:
merged_df.columns

In [ ]:
#最终花了1个小时跑完，为了节约时间将把这段代码的输出结果保存
# stored_col = merged_df[['calendar_day','article_id','days_since_last_promo']].copy()
# stored_col.to_csv('days_since_last_promo.csv', index=False)
# stored_col_test = merged_df_test[['calendar_day','article_id','days_since_last_promo']].copy()
# stored_col_test.to_csv('days_since_last_promo_test.csv', index=False)

In [ ]:
sales = merged_df.copy() 
sales = sales.drop(['article_id','calendar_day','promo_units','scanback','article_desc','category_x','category_y','brand','segment','brandtype','week','competitor','sales_amount','gross_profit','promo_sales','fiscal_week_start_date','promo_ends'],axis=1)
sales_test = merged_df_test.copy()
sales_test = sales_test.drop(['article_id','calendar_day','promo_units','scanback','article_desc','category_x','category_y','brand','segment','brandtype','week','competitor','sales_amount','gross_profit','promo_sales','fiscal_week_start_date','promo_ends'],axis=1)
sales.info()

In [ ]:
sales_dummy = pd.get_dummies(sales,drop_first=True)
sales_dummy_test = pd.get_dummies(sales_test,drop_first= True)

In [ ]:
test_gross_profit = merged_df_test.loc[:,'gross_profit']

sales_amount = np.array(merged_df_test['sales_amount'])
gross_profit = np.array(merged_df_test['gross_profit'])
promo_price = np.array(merged_df_test['promo_price'])
gst_flag = np.array(merged_df_test['gst_flag'])
sales_amount_ex_gst = sales_amount/(1+0.1*gst_flag)
cogs = (sales_amount_ex_gst - gross_profit)
sales_units = merged_df_test['sales_units']

ind_cogs = list(map(lambda cog, units : cog / units if units != 0 else 0, cogs, sales_units))

## Model 1 :Elastic Net

In [ ]:
train_y = sales_dummy.loc[:,'sales_units'].copy()
train_X = sales_dummy.copy().drop(['sales_units','competitor_shelf_price','competitor_total_price'],axis=1)

elastic_net = ElasticNetCV(l1_ratio=[0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.99,1], cv=5)
elastic = elastic_net.fit(train_X,train_y)

In [ ]:
train_X.shape #(1104605, 42)

In [ ]:
elastic.l1_ratio_

In [ ]:
# 看各变量的重要程度
betas = elastic_net.coef_
predictors = list(train_X.columns.values[:])
indicies = np.argsort(np.abs(betas))[-20:]
top_predictors = np.array(predictors)[indicies]
top_betas = betas[indicies]

plt.figure(figsize = (30, 15))
plt.barh(top_predictors, top_betas, alpha=0.5, edgecolor='black')
sns.despine()
plt.xlabel('Beta coefficient')
plt.title('Freature importance for Elastic Net', fontsize = 20)
plt.savefig('elastic_var_importance.png')

In [ ]:
test_y = sales_dummy_test.loc[:,'sales_units'].copy()
test_X = sales_dummy_test.copy().drop(['sales_units','competitor_shelf_price','competitor_total_price'],axis=1)


#test RMSE
result = elastic.predict(test_X)
print(mean_squared_error(test_y,result, squared= False))


In [ ]:
# Train RMSE
result_training = elastic.predict(train_X)
print(mean_squared_error(train_y,result_training, squared= False))

In [ ]:
regular_sales_m1 = np.array(result) * sell_price
promo_sales_m1 = np.array(result) * 0.1 * np.array(merged_df_test['is_promo']) * promo_price
sales_amount_m1 = regular_sales_m1 + promo_sales_m1
gross_profit_test_m1 = sales_amount_m1 - (ind_cogs * np.array(result))
print(mean_squared_error(gross_profit_test_m1 ,gross_profit, squared= False))

In [ ]:
residual_train = train_y - result_training
residual_test = test_y - result

In [ ]:
sns.scatterplot(x = result_training, y = residual_train, label = 'Training set', color = 'blue', s = 60)
sns.scatterplot(x = result, y = residual_test, label = 'Test set',color = 'red', s = 60)

plt. axhline(0, color = 'black', linestyle = '--',linewidth = 1)
plt.xlabel('Sales units (predicted value)')
plt.ylabel('Residuals')
plt.title('residual plot for Elastic net')
plt.legend()
plt.savefig('residual_plot_elastic_net.png')

In [ ]:
def adjusted_r2_score(y_true, y_pred, n_features):
    n = len(y_true)
    r2 = r2_score(y_true, y_pred)
    adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - n_features - 1)
    return adjusted_r2

In [ ]:
#Calculate adj R2 score
n_feature = np.count_nonzero(betas)
adj_r2 = adjusted_r2_score(test_y,result,n_feature)
print(adj_r2)

## Model 2: Random forest

In [ ]:
tree = RandomForestRegressor(criterion= 'squared_error', min_samples_leaf=10, max_features=100, random_state= 0)

parameters = {'n_estimators' : np.arange(100,200,10),
              'min_samples_leaf': np.arange(100, 1000),
              'max_features': np.arange(1, len(train_X.columns)-1)}


random_search = RandomizedSearchCV(RandomForestRegressor(criterion='squared_error', random_state=0), 
                                   parameters, n_iter=1, cv=5, 
                                   scoring='neg_mean_squared_error', random_state=0)

random_search.fit(train_X,train_y)

In [ ]:
params = random_search.best_params_
params

In [ ]:
rf = RandomForestRegressor(random_state=0,**params)
model2 = rf.fit(train_X,train_y)
result_rf = model2.predict(test_X)
#test RMSE
print(mean_squared_error(test_y,result_rf, squared= False))

In [ ]:
#train RMSE
result_rf_train = model2.predict(train_X)
print(mean_squared_error(train_y,result_rf_train, squared= False))

In [ ]:
features = train_X.columns
importances = pd.Series(model2.feature_importances_, index=features).sort_values()

plt.barh(range(len(importances[-20:])), importances[-20:], tick_label=importances.index[-20:])
plt.title('Random Forest Feature Importance')
plt.xlabel('Importance'); plt.ylabel('Feature')
plt.tight_layout()
plt.savefig('importances_rf.png')

In [ ]:
regular_sales_m2 = np.array(result_rf) * sell_price
promo_sales_m2 = np.array(result_rf) * 0.1 * np.array(merged_df_test['is_promo']) * promo_price
sales_amount_m2 = regular_sales_m2 + promo_sales_m2
gross_profit_test_m2 = sales_amount_m2 - (ind_cogs * np.array(result_rf))
print(mean_squared_error(gross_profit_test_m2 ,gross_profit, squared= False))

In [ ]:
r2_score(test_y,result_rf)

In [ ]:
residual_train_rf = train_y - result_rf_train
residual_test_rf = test_y - result_rf

In [ ]:
sns.scatterplot(x = result_rf_train, y = residual_train_rf, label = 'Training set', color = 'blue', s = 60)
sns.scatterplot(x = result_rf, y = residual_test_rf, label = 'Test set',color = 'red', s = 60)

plt. axhline(0, color = 'black', linestyle = '--',linewidth = 1)
plt.xlabel('Sales units (predicted value)')
plt.ylabel('Residuals')
plt.title('residual plot for Random Forest')
plt.legend()
plt.savefig('residual_plot_random_forest.png')

## Model 3: XGBoosting

**Model training with hyperparameter selection**. We can use GridSearchCV to select the best parameters. The most important hyperparameters to tune is: 
* `learning_rate`: learning rate
* `n_estimators`: number of classifiers to add
* `max_depth`: maximum depth of each decision tree model
* `subsample`: Controls the fraction of the total training set to use to train each individual tree. By randomly sampling part of the data for each tree, it adds randomness into the model, which can help improve its robustness and prevent overfitting.
* `colsample_bytree`: controls the fraction of features used for building each tree

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
import matplotlib.pyplot as plt

# load dataset 
# One-hot encode the categorical variables

# X = merged_df[['brandtype','category_subcategory','rank','day_of_week','avg_price', 'avg_promo_price', 'is_promo', 'promo_power', 'num_skus_on_sale']]
# X_encoded = pd.get_dummies(X, columns=['brandtype', 'category_subcategory','day_of_week','is_promo',])
# y = merged_df['sales_units']

# X_test = pd.get_dummies(df_test_merged[['brandtype', 'category_subcategory', 'rank', 'day_of_week', 'avg_price', 'avg_promo_price', 'is_promo', 'num_skus_on_sale']])
# X_test = X_test.reindex(columns=X_encoded.columns, fill_value=0)  # Ensure test set has the same features as the training set
# y_test = df_test_merged['sales_units']


In [ ]:
# Define the model
xgb_model = XGBRegressor(random_state=0,objective='reg:squarederror',verbosity=2) 

# Define the parameter grid
param = {
    'max_depth': [2,3,4],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.2, 0.5],
#     'subsample': [0.8, 1],
#     'colsample_bytree': [0.8, 1]
}

# Define the scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)


#------------- Set up GridSearchCV -------- # took me fking 4 hrs 
# grid_search = GridSearchCV(estimator=xgb_model, param_grid=param, cv = 5, scoring=mse_scorer, verbose=3, n_jobs=-1)

# # Fit the GridSearchCV to find the best parameters
# grid_search.fit(train_X, train_y)


# ------------- Randomized search -----------#
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'max_depth': [4, 5,6],  # 最大深度
   'min_child_weight': [ 3, 7, 10],  # 增加最小子节点所需的数据量
    'subsample': [0.7, 0.9],  # 行采样比例
    'colsample_bytree': [0.5, 0.9],  # 列采样比例
    'learning_rate': [0.01, 0.02, 0.05],  # 学习率
    'n_estimators': [300,500,700]  # 增加树的数量
}


random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions, n_iter=50, scoring=mse_scorer, cv=5, verbose=3, random_state=0, n_jobs=-1)


random_search.fit(train_X, train_y)





# Display the best parameters and best score
# print("Best parameters found:", grid_search.best_params_)
# print("Best mean squared error:", -grid_search.best_score_)


print("Best parameters found:", random_search.best_params_)
print("Best mean squared error:", -random_search.best_score_)

In [ ]:
# Make predictions
# Retrieve the best estimator (model) found by GridSearchCV
best_model = random_search.best_estimator_

# Evaluate the model using the test set
predictions = best_model.predict(test_X)
mse_test = mean_squared_error(test_y, predictions)
print("Mean Squared Error on the Test Set:", mse_test)

In [ ]:
importance = best_model.feature_importances_
features = pd.Series(importance, index=train_X.columns).sort_values(ascending=True).head(20)

# Plotting
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
features.plot(kind='barh')
plt.title('Top 20 Important Features')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.savefig('fm.png', bbox_inches='tight')
plt.show()


In [ ]:
regular_sales_m3 = np.array(predictions) * sell_price
promo_sales_m3 = np.array(predictions) * 0.1 * np.array(merged_df_test['is_promo']) * promo_price
sales_amount_m3 = regular_sales_m3 + promo_sales_m3
gross_profit_test_m3 = sales_amount_m3 - (ind_cogs * np.array(predictions))
print(mean_squared_error(gross_profit_test_m3 ,gross_profit, squared= False))

In [ ]:
import numpy as np
from sklearn.metrics import r2_score

# Assuming `xgb_model` is your trained XGBoost model
# `X_test` is your test features, `y_test` is your test target

# Predict using the trained model

# Calculate R-squared
r_squared = r2_score(test_y, predictions)

# Number of observations
n = test_X.shape[0]

# Number of features (predictors)
p = test_X.shape[1]

# Calculate Adjusted R-squared
adjusted_r_squared = 1 - (((1 - r_squared) * (n - 1)) / (n - p - 1))

print("R-squared:", r_squared)
print("Adjusted R-squared:", adjusted_r_squared)

In [ ]:
# Calculate residuals
residuals = test_y - predictions

# Plotting the residuals
plt.figure(figsize=(10, 6))
plt.scatter(predictions, residuals, color='blue', alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot of test set')
plt.savefig('residual.png')
plt.show()